In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import math

data available at https://www.kaggle.com/competitions/store-sales-time-series-forecasting/leaderboard upon login.

In [2]:
train_csv=pd.read_csv("~/Downloads/store-sales-time-series-forecasting/new_train.csv",parse_dates=True)

train_csv["date"]=pd.to_datetime(train_csv["date"])
#train_csv["date"]=train_csv["date"].apply(lambda x: datetime.timestamp(x))

stores_csv=pd.read_csv("~/Downloads/store-sales-time-series-forecasting/stores.csv")

holiday_csv=pd.read_csv("~/Downloads/store-sales-time-series-forecasting/holidays_events.csv",parse_dates=True)

holiday_csv["date"]=pd.to_datetime(holiday_csv["date"])
#holiday_csv["date"]=holiday_csv["date"].apply(lambda x: datetime.timestamp(x))

oil_csv=pd.read_csv("~/Downloads/store-sales-time-series-forecasting/oil.csv")

oil_csv["date"]=pd.to_datetime(oil_csv["date"])

trans_csv=pd.read_csv("~/Downloads/store-sales-time-series-forecasting/transactions.csv")

trans_csv["date"]=pd.to_datetime(trans_csv["date"])
#trans_csv["date"]=trans_csv["date"].apply(lambda x: datetime.timestamp(x))

test_csv=pd.read_csv("~/Downloads/store-sales-time-series-forecasting/test.csv")

In [3]:
#### Generating a time index from 2013-01-01 to 2017-08-31 ################
idx = pd.date_range('1/1/2013', periods=1704, freq='D')

In [4]:
#### Generating a time index from 2013-01-01 to 2017-08-15 for train set only################

idx_train = pd.date_range('1/1/2013', periods=1688, freq='D')

# ADDING MISSING OIL PRICES 

In [5]:
######## creating a new oil dataframe ###############
new_oil=pd.DataFrame(columns=["date","price"])

In [6]:
####### setting the date column from 2013-01-01 to 2017-08-31 #############
new_oil["date"]=idx

In [7]:
########### adjoining intermediate missing values ############
for i in range(len(idx)):
    x=new_oil.iloc[i,0]
    if x in oil_csv["date"].values:
        new_oil.iloc[i,1]=oil_csv[oil_csv["date"]==x].iloc[0,1]

In [8]:
########### setting the first missing value at the start of the dataframe ############
new_oil.iloc[0,1]=93.14

In [9]:
########### replacing missing values with the mean of previous two values ############

for i in range(1,len(new_oil)):
    if math.isnan(new_oil.iloc[i,1]):
        new_oil.iloc[i,1]=(new_oil.iloc[i-1,1]+new_oil.iloc[i-2,1])/2

In [10]:
######### setting the datatype for price ##########
new_oil["price"]=new_oil["price"].astype(float)

In [11]:
######### saving the new dataframe ################
new_oil.to_csv("~/Downloads/store-sales-time-series-forecasting/new_oil.csv",index=False)

# TRANSACTIONS

In [ ]:
######checking which dates are missing from trans_csv#########
for x in idx_train:
    if x not in trans_csv["date"].values:
        print(x)

In [ ]:
store_no=stores_csv["store_nbr"].values

In [ ]:
############# checking if all dates have entries for all stores###########
for x in idx_train:
    missing=[]
    for y in store_no:
        if y not in trans_csv[trans_csv["date"]==x].iloc[:,1].values:
            missing.append(y)
    if len(missing)>0:
        print(x,missing)

# TRAIN SET

In [ ]:
######### constructing a list of products #################
product=train_csv["family"].unique()

In [ ]:
########## checking if all dates are present in train ##########
missing_dates=[]
for x in idx_train:
    if x not in train_csv["date"].values:
        missing_dates.append(x.date())
print(missing_dates)

In [ ]:
############# checking if all dates have entries for all stores #############
for x in idx_train:
    missing=[]
    for y in store_no:
       # print(train_csv[train_csv["date"]==x])
        if y not in train_csv[train_csv["date"]==x].iloc[:,1].values:
            missing.append(y)
    if len(missing)>0:
        print(x,missing)

In [ ]:
#############check if all product family is there for all dates###############
for x in idx_train:
    for y in store_no:
        check1=train_csv["date"]==x
        check2=train_csv["store_nbr"]==y
        var=train_csv[check1 & check2]
        var2=var.values
        missing=[]
        for z in product:
            if z not in var2:
                missing.append(z)
        if len(missing)>0:
            print(x,y,missing)

# ADDING MISSING VALUES TO TRAIN

In [ ]:
####### dropping the id column from train #############
train_csv.drop(columns="id",inplace=True)

In [ ]:
########### constructing an array consisting of missing values only ##############            
append_train=[]
for x in missing_dates:
    for y in store_no:
        for z in product:
            #check1=train_csv["date"]==x
            check2=train_csv["store_nbr"]==y
            check3=train_csv["family"]==z
            check4=train_csv["date"]==pd.to_datetime(x-timedelta(days=1))
            check5=train_csv["date"]==pd.to_datetime(x+timedelta(days=1))
            app_prom=(train_csv[check4 & check2 & check3].iloc[0,4]+train_csv[check5 & check2 & check3].iloc[0,4])/2
            app_sales=(train_csv[check4 & check2 & check3].iloc[0,3]+train_csv[check5 & check2 & check3].iloc[0,3])/2
            app=[pd.to_datetime(x),y,z,app_sales,app_prom]
            append_train.append(app)

  

In [ ]:
########### constructing a dataframe consisting of missing values only ##############            
pd.DataFrame(append_train,columns=["date","store_nbr","family","sales","onpromotion"])

In [ ]:
########## appending the missing values to original dataframe and naming it new_train ################
new_train=train_csv.append(pd.DataFrame(append_train,columns=["date","store_nbr","family","sales","onpromotion"]),\
                           ignore_index=True)

In [ ]:
######### sorting new_train ##############
new_train=new_train.sort_values(by=["date","store_nbr"],ignore_index=True)

In [ ]:
########### saving as new_train_csv #############
new_train.to_csv("~/Downloads/store-sales-time-series-forecasting/new_train.csv",header=new_train.columns,index=False,date_format='%Y-%m-%d %H:%M:%S')

# HOLIDAYS

In [ ]:
holiday_csv=pd.DataFrame.drop(holiday_csv,columns=["locale","description"])

In [ ]:
holiday_csv

In [ ]:
holiday_csv["date"]=pd.to_datetime(holiday_csv["date"].values)

In [ ]:
####### constructing an array of holidays and adding non holidays as working day############
list=[]
for x in idx:
    if x in holiday_csv["date"].values:
        list.append(holiday_csv[holiday_csv["date"]==pd.to_datetime(x)].values[0])
    else:
        list.append([pd.to_datetime(x),'Work Day','Ecuador',False])

In [ ]:
####### saving the new dataframe as new_holiday ################
X=pd.DataFrame(list,columns=["date","type","locale_name","transferred"])

X.to_csv("~/Downloads/store-sales-time-series-forecasting/new_holiday.csv",index=False)